In [88]:
train.head()

,etoiles,n_avis,date_avis,text_total,codes agrégés,c_good_value,c_bad_value,c_good_efficacy,c_bad_efficacy,c_good_comm,c_bad_comm,text_sent_noPunct,text_clean_neg,text_clean_post,text_clean_pos,text_clean_pos_whole,text_clean_neg_whole
NaN,3,3.0,2022-07-26 10:10:19,pour l'instant toujours en attente. pour l'ins...,bad:inefficace,0,0,0,1,0,0,"[pour l' instant toujours en attente, pour l' ...","[pour l' instant toujours en attente, pour l' ...",[],[],,pour l' instant toujours en attente pour l' in...
NaN,3,1.0,2022-04-11 14:14:30,intérêts trop élevés.,none,0,0,0,0,0,0,[intérêts trop élevés],[intérêts trop élevés],[],[],,intérêts trop élevés
NaN,2,3.0,2023-06-07 16:57:39,"c'était les meilleurs, mais c'était avant. deu...",bad:communication;bad:inefficace,0,0,0,1,0,1,"[c' était les meilleurs mais c' était avant, d...","[c' était les meilleurs mais c' était avant, d...",[pour moi fortunéo était la meilleure banque m...,[pour moi fortunéo était la meilleure banque m...,pour moi fortunéo était la meilleure banque ma...,c' était les meilleurs mais c' était avant deu...
NaN,1,1.0,2023-09-08 13:30:04,vous n'aidez pas les gens. vous n'aidez pas le...,none,0,0,0,0,0,0,"[vous n' aidez pas les gens, vous n' aidez pas...","[vous n' aidez pas les gens, vous n' aidez pas...",[],[],,vous n' aidez pas les gens vous n' aidez pas l...
NaN,1,2.0,2023-02-10 20:32:37,de voyage à l'étranger mon conseillé de…. de v...,bad:clôture_abusive,0,0,0,1,0,0,"[de voyage à l' étranger, mon conseillé de, de...",[de voyage à l' étranger mon conseillé de la b...,"[de voyage à l' étranger, mon conseillé de]","[de voyage à l' étranger, mon conseillé de]",de voyage à l' étranger mon conseillé de,de voyage à l' étranger mon conseillé de la ba...


In [ ]:
# créer une version du texte listé par phrases de plus de un mot et sans ponctuation
train['text_sent_noPunct'] = ""
def create_review_sent_noPunct(df)
for i, review in enumerate(df.text_total):
    # Reconstruct sentences without punctuation
    review = nlp(review)
    sentences_without_punctuation = []
    for sent in review.sents:
        # Filter out tokens that are punctuation
        tokens_without_punctuation = [token.text for token in sent if not token.is_punct]
        if len(tokens_without_punctuation) >1:   
            # Join the tokens to form a new sentence
            new_sentence = " ".join(tokens_without_punctuation)
            sentences_without_punctuation.append(new_sentence)
    df['text_sent_noPunct'].iloc[i] = sentences_without_punctuation

# créer deux colones: une pour les phrases positives, l'autre pour les négatives
import pandas as pd
import numpy as np
import spacy
from transformers import pipeline,AutoTokenizer, TFCamembertForSequenceClassification
import sentencepiece

# importing spacy model
nlp = spacy.load("fr_core_news_lg")

# importing  tokenizer and Camembert classifier
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine")
model = TFCamembertForSequenceClassification.from_pretrained("tblard/tf-allocine")
classifier=  pipeline("text-classification", model = model, tokenizer= tokenizer)
#tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512,'return_tensors':'pt'}



def creating_cols_pos_neg_sents(df):
    df['text_clean_neg'] = ""
    df['text_clean_pos'] = ""
    for i, sentence_list in enumerate(df['text_sent_noPunct']):
        pos_sent, neg_sent = [],[]
        print("new message")
        for sentence in sentence_list:
            sentence = sentence[:512] # nombre de mots max pour Camembert
            try:
                temp = classifier(sentence)[0]['label']
            except Exception as e:
                print(f"Erreur lors du traitement du texte {sentence}. Erreur: {e}")
            if temp == "POSITIVE":
                pos_sent.append(sentence)
            elif temp == "NEGATIVE":
                neg_sent.append(sentence)
        df['text_clean_neg'].iloc[i] = neg_sent
        df['text_clean_pos'].iloc[i] = pos_sent
    
    df["text_clean_pos_whole"] = ""
    for i,sentence_list in enumerate(df.text_clean_pos):
        df["text_clean_pos_whole"].iloc[i] = " ".join([sent for sent in sentence_list])

    df["text_clean_neg_whole"] = ""
    for i,sentence_list in enumerate(df.text_clean_neg):
        df["text_clean_neg_whole"].iloc[i] = " ".join([sent for sent in sentence_list])


creating_cols_pos_neg_sents()

In [ ]:
one_shot_review_vs_whole_ref_bag(bag_of_ref,reviews, y,code_of_interest,thresh, train_test, word_mode)

In [ ]:
def decision_sentiment(sentiment,code_of_interest, sentence_whole):
    if sentiment == True:
        if "good" in code_of_interest:
            if "sentence" in sentence_whole:
                reviews =  "text_clean_sentences_pos"
            elif "whole" in sentence_whole:
                reviews = "text_clean_pos_whole"    
        elif "bad" in code_of_interest:
            if "sentence" in sentence_whole:
                reviews =  "text_clean_sentences_neg "
            elif "whole" in sentence_whole:
                reviews = "text_clean_neg_whole"
    elif sentiment == False:
        if "whole" in sentence_whole:
            reviews = "text_sent_noPunct_whole"
        elif "sentence" in sentence_whole:
            reviews = "text_sent_noPunct_sentences"
    else :
        print("error: no sentiment / granularity found")
    return reviews
            

def core_full_review_vs_whole_ref_bag(bag_of_ref,df, thresh,code_of_interest, sentiment):
    y_pred = []
    reviews = decision_sentiment(sentiment, code_of_interest, "whole")
    df[reviews]
    for message in reviews:
        print (message) 
        sim_score = []
        message = nlp(message)
        sim_score.append(round(bag_of_ref.similarity(message),2))
        allocate_lab(sim_score, y_pred, thresh)
    return y_pred

def core_review_sentence_vs_whole_ref_bag(bag_of_ref,reviews, thresh):
    y_pred = []
    for message in reviews:
        sim_score = []
        message = nlp(message)
        sentences = message.sents
        for sentence in sentences:
            if sentence.has_vector:
                try:
                    temp_sim_score = round(bag_of_ref.similarity(sentence),2)

                    sim_score.append(temp_sim_score)
                except:
                    print("issue with vector")
                    sim_score.append(0)
        allocate_lab(sim_score, y_pred, thresh)
    return y_pred

def core_review_sentence_vs_reference_sentence(bag_of_ref,reviews, thresh):
    y_pred,trigger_ref_sent = [],[]
    for message in reviews:
        sim_score= []
        message = nlp(message)
        review_sentences = message.sents
        sentences_exemples = bag_of_ref.sents
        for review_sentence in review_sentences:
            if review_sentence.has_vector:
                for sentence_exemple in sentences_exemples:
                    if sentence_exemple.has_vector:
                        try:
                            temp_sim_score = round(review_sentence.similarity(sentence_exemple),2)
                            sim_score.append(temp_sim_score)
                        except:
                            print("issue with vector")
                            sim_score.append(0.0)
        print('sim score', sim_score)
        best_sim_score =  max(sim_score)
        list_trigger_ref_sents = [sent for sent in sentences_exemples]
        trigger_ref_sent.append(list_trigger_ref_sents[sim_score.index(best_sim_score)])
        allocate_lab(sim_score, y_pred, thresh)
    return y_pred, trigger_ref_sent

### toujours le meme f1 score??? Vaut pour les deux étalons est ce que sim score est toujours sous le seuil?
def core_review_sentence_vs_ref_étalon(etalon,reviews, thresh):
    y_pred = []
    etalon = nlp(etalon)
    for message in reviews:
        sim_score= []
        message = nlp(message)
        sentences = message.sents
        for sentence in sentences:
            if sentence.has_vector:
                try:
                    temp_sim_score = round(etalon.similarity(sentence),2)
                    sim_score.append(temp_sim_score)
                    #print("sim score review sentence:", temp_sim_score)
                except:
                    print("issue with vector")
                    sim_score.append(0)
        allocate_lab(sim_score, y_pred, thresh)
    return y_pred
        
def core_full_review_vs_étalon(etalon,reviews, thresh):
    y_pred = []
    etalon = nlp(etalon)
    for message in reviews:
        sim_score = []
        message = nlp(message)
        sim_score.append(round(etalon.similarity(message),2))
        print("sim score review sentence:", sim_score[-1])
        allocate_lab(sim_score, y_pred, thresh)
    return y_pred